In [198]:
import spacy
from pdfminer.pdfinterp import PDFResourceManager,PDFPageInterpreter
from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams
from pdfminer.pdfpage import PDFPage
from io import BytesIO

In [199]:
import spacy
from pdfminer.pdfinterp import PDFResourceManager,PDFPageInterpreter
from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams
from pdfminer.pdfpage import PDFPage
from io import BytesIO
import re
from spacy.matcher import Matcher
from nltk.corpus import stopwords

class Resume_parser:
    def __init__(self,path):
        self.path=path
        self.text=self.pdf_to_text()
        self.name=self.extract_name()
        self.mobile=self.extract_mobile_number()
        self.email=self.extract_email()
        self.skills=self.extract_skills()
        self.education=self.extract_education()
        self.link=self.extract_project_link()
    def pdf_to_text(self):
        manager = PDFResourceManager()
        retstr = BytesIO()
        layout = LAParams(all_texts=True)
        device = TextConverter(manager, retstr, laparams=layout)
        filepath = open(self.path, 'rb')
        interpreter = PDFPageInterpreter(manager, device)
        for page in PDFPage.get_pages(filepath, check_extractable=True):
            interpreter.process_page(page)
            text = retstr.getvalue()
        filepath.close()
        device.close()
        retstr.close()
        return text.decode()
    def extract_name(self):
        #Load Pretrain data
        nlp = spacy.load('en_core_web_sm')
        # initialize matcher with a vocab
        matcher = Matcher(nlp.vocab)
        nlp_text = nlp(self.text)
        # First name and Last name are always Proper Nouns
        pattern = [{'POS': 'PROPN'}, {'POS': 'PROPN'}]
        matcher.add("NAME",None,pattern)
        matches = matcher(nlp_text)
        for match_id, start, end in matches:
            span = nlp_text[start:end]
            return span.text
    def extract_mobile_number(self):
        phone = re.findall(re.compile(r'(?:(?:\+?([1-9]|[0-9][0-9]|[0-9][0-9][0-9])\s*(?:[.-]\s*)?)?(?:\(\s*([2-9]1[02-9]|[2-9][02-8]1|[2-9][02-8][02-9])\s*\)|([0-9][1-9]|[0-9]1[02-9]|[2-9][02-8]1|[2-9][02-8][02-9]))\s*(?:[.-]\s*)?)?([2-9]1[02-9]|[2-9][02-9]1|[2-9][02-9]{2})\s*(?:[.-]\s*)?([0-9]{7})(?:\s*(?:#|x\.?|ext\.?|extension)\s*(\d+))?'),self.text)
        if phone:
            number = ''.join(phone[0])
            if len(number) > 10:
                return '+' + number 
            else:
                return number
    def extract_email(self):
        email = re.findall("([^@|\s]+@[^@]+\.[^@|\s]+)",self.text)
        if email:
            try:
                return email[0].split()[0].strip(';')
            except IndexError:
                return None
    def extract_skills(self):
        nlp = spacy.load('en_core_web_sm')
        nlp_text = nlp(self.text)
        noun_chunks = nlp_text.noun_chunks

        # removing stop words and implementing word tokenization
        tokens = [token.text for token in nlp_text if not token.is_stop]

        # extract values
        skills =["node.js","deep learning",'php,'"mongodb","nodejs","redis","ajax"]

        skillset = []

        # check for one-grams (example: python)
        for token in tokens:
            if token.lower() in skills:
                skillset.append(token) 
        # check for bi-grams and tri-grams (example: machine learning)
        for token in noun_chunks:
            token = token.text.lower().strip()
            if token in skills:
                skillset.append(token)

        return [i.capitalize() for i in set([i.lower() for i in skillset])]
    def extract_education(self):
        nlp = spacy.load('en_core_web_sm')

        # Grad all general stop words
        STOPWORDS = set(stopwords.words('english'))

        # Education Degrees
        EDUCATION = [

                'BE','B.E.', 'B.E', 'BS', 'B.S', 
                'ME', 'M.E', 'M.E.', 'MS', 'M.S', 
                'BTECH', 'B.TECH', 'M.TECH', 'MTECH', 
                'SSC', 'HSC', 'CBSE', 'ICSE', 'X', 'XII'
            ]
        nlp_text = nlp(self.text)

        # Sentence Tokenizer
        nlp_text = [sent.string.strip() for sent in nlp_text.sents]

        edu = {}
        # Extract education degree
        for index, text in enumerate(nlp_text):
            for tex in text.split():
                # Replace all special symbols
                tex = re.sub(r'[?|$|.|!|,|(|)]', r'', tex)
                if tex.upper() in EDUCATION and tex not in STOPWORDS:
                    if index+1<len(nlp_text):
                        edu[tex] = text + nlp_text[index+1]
                    else:
                        edu[tex] = text + nlp_text[index]
        # Extract year
        education = []
        for key in edu.keys():
            year = re.search(re.compile(r'(((20|19)(\d{2})\s*-\s*(20|19)(\d{2})))|((20|19)(\d{2}))'), edu[key])
            if year:
                education.append((key, ''.join(year[0])))
            else:
                education.append(key)
        return education
    
    def extract_project_link(self):
        nlp = spacy.load('en_core_web_sm')
        Link=["https","http"]
        nlp_text=nlp(self.text)
        nlp_text=[sent.string.strip() for sent in nlp_text.sents]
        project_link=[]
        for link in nlp_text:
            for protocol in link.split(':'):
                if protocol.lower() in Link:
                    project_link.append(link)
        return project_link
rae=Resume_parser("./Resume (2).pdf")

In [200]:
print(rae.name)
print(rae.email)
print(rae.mobile)
print(rae.skills)
print(rae.education)
for link in rae.link:
    print(link)


Mahajan Sahil
sahilmahajanstar@gmail.com
+919657859876
['Node.js', 'Ajax', 'Redis', 'Deep learning']
[('BE', '2017 - 2021'), ('XII', '2017'), ('X', '2015'), 'HSC']
https://www.linkedin.com/in/timbuchalka/?originalSubdomain=au
https://drive.google.com/drive/folders/1Gm3NBturTH4VEPoxKSZIirdl5OBsEckX?usp=shar
https://github.com/sahilmahajanstar/FarmWorkDiary/blob/master/Farm.py
https://drive.google.com/drive/folders/1C4p62GqJlFmpRVup2dWa3J4aI9q47JwC?usp=sha
